# **ONLY YEAR 2017, Replica of RUSBoost previous work**

## Python Environment

### Import Python libraries

In [ ]:
# Data libraries
#from google.colab import drive
import os
import pandas as pd
import calendar
import numpy as np
import gc
import math
from operator import itemgetter

# Plotting
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from seaborn import violinplot, boxenplot

# Model training
from sklearn.model_selection import cross_validate
from sklearn import metrics, preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
#from imblearn.ensemble import RUSBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import base
import joblib
    # Keras
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Lambda
    # Tensorflow
import tensorflow as tf

# Statistical tests
from scipy.stats import shapiro

### SET ENVIRONMENT VARIABLES

In [ ]:
# WORKING FOLDER: /content/local/Traffic/data or /content/drive/MyDrive/Traffic/data
ROOT = '/content/local/Traffic/data'
# PREDICTION HORIZON
PREDICTED_MINUTES_AHEAD = 5
if (PREDICTED_MINUTES_AHEAD%5 != 0): raise ValueError("Invalid prediction horizon. Must be multiple of 5 minutes")
INPUT_MINUTES_BEFORE = 45
if (INPUT_MINUTES_BEFORE%5 != 0): raise ValueError("Invalid input minutes span. Must be multiple of 5 minutes")

# **LOAD DATA**

## DATA [ LINK_ID, TT_ARR, LOS_DEP ] (Year 2017)

In [ ]:


training_mon = []
training_week = []
training_fri = []
training_weeknd = []

directory = '/content/drive/MyDrive/Traffic/data/processed/'
for filename in os.listdir(directory):
    if "2017" not in filename:
        continue
    f = os.path.join(directory, filename)
    df=pd.read_csv(f)
    print(f'{filename} {df.shape[0]/(60*24*6)}')
    df["date"] = pd.to_datetime(df["date"], format='%d-%b-%Y %H:%M:%S')
    df = df.set_index("date")
    Y = df.shift(-PREDICTED_MINUTES_AHEAD, freq='min')[['link_id', 'LOS_dep']]
    # Training set contains all data
    X = df[['link_id', 'tt_arr']]
    for weekday in range(0,7):
        # Mon
        if weekday == 0:
            training_mon.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        # Tue-Wed-Thu
        elif weekday in [1,2,3]:
            training_week.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        elif weekday == 4:
            training_fri.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        elif weekday in [5,6]:
            training_weeknd.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )

del df,X,Y
training_df_mon = pd.concat(training_mon)
training_df_week = pd.concat(training_week)
training_df_fri = pd.concat(training_fri)
training_df_weeknd = pd.concat(training_weeknd)

del training_mon,training_week,training_fri,training_weeknd

## DATA [ LINK_ID, TT_ARR, TT_ARR-5, TT_ARR-10, ..., TT_ARR-N, LOS_DEP ] (Year 2017)

In [ ]:


training = []

directory = '/content/drive/MyDrive/Traffic/data/processed/'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    df=pd.read_csv(f)
    print(f'{filename} {df.shape[0]/(60*24*6)}')
    df["date"] = pd.to_datetime(df["date"], format='%d-%b-%Y %H:%M:%S')
    df = df.set_index("date")
    Y = df.shift(-PREDICTED_MINUTES_AHEAD, freq='min')[['link_id', 'LOS_dep']]
    # Training set
    X = df[['link_id', 'tt_arr']]
    for offset in range(1,int((INPUT_MINUTES_BEFORE)/5)+1):
        X = pd.merge(X, df.shift(offset*5, freq='min')[['link_id', 'tt_arr']], on=['date', 'link_id'], how='inner', suffixes=[None, f'-{int(offset*5)}'])
    for weekday in range(0,7):
        # Mon
        if weekday == 0:
            training_mon.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        # Tue-Wed-Thu
        elif weekday in [1,2,3]:
            training_week.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        elif weekday == 4:
            training_fri.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )
        elif weekday in [5,6]:
            training_weeknd.append( pd.merge(X, Y[(Y.index.weekday == weekday)], on=['date', 'link_id'], how='inner') )

del df,X,Y
training_df_mon = pd.concat(training_mon)
training_df_week = pd.concat(training_week)
training_df_fri = pd.concat(training_fri)
training_df_weeknd = pd.concat(training_weeknd)

del training_mon,training_week,training_fri,training_weeknd

# **RUSBoost**

### *Algorithms*

**Algorithm implementation M2**

In [ ]:
def labelToIndex(cl,clf):
    return clf.labelDict[cl]

def indexToLabel(i,clf):
    return clf.classes[i]

class RUSBoostClassifier_:
    
    def __init__(self,base_estimator=None,n_estimators=50,learning_rate=1.0):
        print("M2 Implementation of RUSBoost")
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        if base_estimator == None:
            base_estimator = DecisionTreeClassifier(max_depth=1)
        self.base_estimator = base_estimator
        self.estimator_errors_ = []
        self.observation_weights_ = {}
        
    def classes_(self):
        return self.classes

    def fit(self, X: pd.DataFrame, y: pd.DataFrame):
        """
        param X: DataFrame that contains the features with shape (n_samples, )
        param y: DataFrame with the labels
        """        
        
        # Class labels mapping to indices
        self.createLabelDict(np.unique(y))
        k = len(self.classes)
        # `undersampling_n` elements to sample from each class, equal #samples minority class
        undersampling_n = min(y.value_counts())

        # Initialize observation weights as 1/(N*(k-1)) where N is total `n_samples` and k is the numebr of classes
        N = X.shape[0]
        B = N*(k-1)
        D = {epoch: np.full(k-1,1/B) for epoch in X.index}

        # Get whole dataset samples to later calculate the weighting factors on every iteration
        iTL = np.vectorize(labelToIndex)
        y_indices = iTL(y.values,self)
        
        # M iterations (#WeakLearners)
        for m in range(self.n_estimators):

        # 1) Random UnderSampling
            df_ = pd.DataFrame()
            for label_ in self.classes:
                df_ = pd.concat([ df_, X.loc[[ y==label_ ]].sample(undersampling_n, replace=False) ])

            
            # Training data initalization
            X_ = df_.values
            y_ = y.loc[df_.index].values
            D_ = np.sum([D[epoch] for epoch in df_.index], axis=-1)

        # 2) WeakLearner training
            Gm = base.clone(self.base_estimator).\
                            fit(X_,y_,sample_weight=D_).predict_proba
            self.models.append(Gm)
        
        # 3) Error-rate computation
            predictions_proba = Gm(X.values)
            sum_pseudolosses = 0
            for i, epoch in enumerate(D.keys()):
                k_index = 0
                for cl in range(k):
                    if cl != y_indices[i]:
                        sum_pseudolosses += D[epoch][k_index]*(1-predictions_proba[i,y_indices[i]]+predictions_proba[i,cl])
                        k_index += 1

            error = 0.5 * sum_pseudolosses
            self.estimator_errors_.append(error)
        
        # 4) WeakLearner weight for ensemble computation
            BetaM = error/(1-error)
            self.models[m] = (BetaM,Gm)

        # 5) Observation weights update for next iteration with weights normalization
            norm_ = 0
            for i, epoch in enumerate(D.keys()):
                k_index = 0
                for cl in range(k):
                    if cl != y_indices[i]:
                        w_ = 0.5*(1+predictions_proba[i,y_indices[i]]-predictions_proba[i,cl])
                        D[epoch][k_index] *= BetaM**(self.learning_rate*w_)
                        norm_ += D[epoch][k_index]
                        k_index += 1
            for epoch in D.keys():
                D[epoch] /= norm_
        
        self.observation_weights_ = D
        
        return self
            
    def createLabelDict(self,classes):
        self.labelDict = {}
        self.classes = classes
        for i,cl in enumerate(classes):
            self.labelDict[cl] = i
    
    def predict(self,X):
        sum_model_hypothesis = np.sum(np.stack([-np.log(Bm)*Gm(X) for Bm,Gm in self.models], axis=-1), axis=-1)
        iTL = np.vectorize(indexToLabel)            
        return iTL(np.argmax(sum_model_hypothesis,axis=1),self)


In [ ]:
for LINK in range(5,11):
    print(f"LINK {LINK}")
    other_links = [5,6,7,8,9,10]; other_links.remove(LINK)
    training_df_link = training_df.copy()
    training_df_link = training_df_link[training_df_link['link_id'] == LINK]

    for other_link in other_links:
        training_df_link = pd.merge(training_df_link, training_df[training_df['link_id'] == other_link].filter(regex=("tt_arr.*")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])
    
    X = training_df_link.filter(regex=("tt_arr.*")).values
    y = training_df_link['LOS_dep'].values
    clf = make_pipeline(preprocessing.StandardScaler(), GaussianNB())

    # 5-fold cross validation
    CV = 5
    scoring = {'recall1': metrics.make_scorer(metrics.recall_score, average = None, labels = [1]), 
        'recall2': metrics.make_scorer(metrics.recall_score, average = None, labels = [2]),
        'recall3': metrics.make_scorer(metrics.recall_score, average = None, labels = [3]),
        'recall4': metrics.make_scorer(metrics.recall_score, average = None, labels = [4]), 
        'recall5': metrics.make_scorer(metrics.recall_score, average = None, labels = [5]),
        'recall6': metrics.make_scorer(metrics.recall_score, average = None, labels = [6])}
    results = cross_validate(clf, X, y, cv=CV, scoring=scoring, return_train_score = False)
    for LOS in range(1,7):
        score = results[f'test_recall{LOS}']
        print(f"\tLOS {LOS}: Recall {round(sum(score)/CV*100, 2)}%")

### **NO FEATURE ENGINEERING M2**

In [ ]:
for LINK in range(5,11):
    print(f"LINK {LINK}")

    features_name = 'tt_arr$'

    print(f"MONDAY")
    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=128), n_estimators=30, learning_rate=0.3)
    X = training_df_mon[training_df_mon['link_id'] == LINK].filter(regex=(features_name))
    y = training_df_mon[training_df_mon['link_id'] == LINK]['LOS_dep']
    # 5-fold cross validation
    CV = 5
    scoring = {'recall1': metrics.make_scorer(metrics.recall_score, average = None, labels = [1]), 
        'recall2': metrics.make_scorer(metrics.recall_score, average = None, labels = [2]),
        'recall3': metrics.make_scorer(metrics.recall_score, average = None, labels = [3]),
        'recall4': metrics.make_scorer(metrics.recall_score, average = None, labels = [4]), 
        'recall5': metrics.make_scorer(metrics.recall_score, average = None, labels = [5]),
        'recall6': metrics.make_scorer(metrics.recall_score, average = None, labels = [6])}
    results = cross_validate(model, X, y, cv=CV, scoring=scoring, return_train_score = False)
    for LOS in range(1,7):
        score = results[f'test_recall{LOS}']
        print(f"\tLOS {LOS}: Recall {round(sum(score)/CV*100, 2)}%")
    
    print(f"TUESDAY-WEDNESDAY-THURSDAY")
    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=196), n_estimators=30, learning_rate=0.3)
    X = training_df_week[training_df_week['link_id'] == LINK].filter(regex=(features_name))
    y = training_df_week[training_df_week['link_id'] == LINK]['LOS_dep']
    # 5-fold cross validation
    CV = 5
    scoring = {'recall1': metrics.make_scorer(metrics.recall_score, average = None, labels = [1]), 
        'recall2': metrics.make_scorer(metrics.recall_score, average = None, labels = [2]),
        'recall3': metrics.make_scorer(metrics.recall_score, average = None, labels = [3]),
        'recall4': metrics.make_scorer(metrics.recall_score, average = None, labels = [4]), 
        'recall5': metrics.make_scorer(metrics.recall_score, average = None, labels = [5]),
        'recall6': metrics.make_scorer(metrics.recall_score, average = None, labels = [6])}
    results = cross_validate(model, X, y, cv=CV, scoring=scoring, return_train_score = False)
    for LOS in range(1,7):
        score = results[f'test_recall{LOS}']
        print(f"\tLOS {LOS}: Recall {round(sum(score)/CV*100, 2)}%")

    print(f"FRIDAY")
    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=128), n_estimators=30, learning_rate=0.3)
    X = training_df_fri[training_df_fri['link_id'] == LINK].filter(regex=(features_name))
    y = training_df_fri[training_df_fri['link_id'] == LINK]['LOS_dep']
    # 5-fold cross validation
    CV = 5
    scoring = {'recall1': metrics.make_scorer(metrics.recall_score, average = None, labels = [1]), 
        'recall2': metrics.make_scorer(metrics.recall_score, average = None, labels = [2]),
        'recall3': metrics.make_scorer(metrics.recall_score, average = None, labels = [3]),
        'recall4': metrics.make_scorer(metrics.recall_score, average = None, labels = [4]), 
        'recall5': metrics.make_scorer(metrics.recall_score, average = None, labels = [5]),
        'recall6': metrics.make_scorer(metrics.recall_score, average = None, labels = [6])}
    results = cross_validate(model, X, y, cv=CV, scoring=scoring, return_train_score = False)
    for LOS in range(1,7):
        score = results[f'test_recall{LOS}']
        print(f"\tLOS {LOS}: Recall {round(sum(score)/CV*100, 2)}%")

    print(f"WEEKEND")
    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=128), n_estimators=30, learning_rate=0.3)
    X = training_df_weeknd[training_df_weeknd['link_id'] == LINK].filter(regex=(features_name))
    y = training_df_weeknd[training_df_weeknd['link_id'] == LINK]['LOS_dep']
    # 5-fold cross validation
    CV = 5
    scoring = {'recall1': metrics.make_scorer(metrics.recall_score, average = None, labels = [1]), 
        'recall2': metrics.make_scorer(metrics.recall_score, average = None, labels = [2]),
        'recall3': metrics.make_scorer(metrics.recall_score, average = None, labels = [3]),
        'recall4': metrics.make_scorer(metrics.recall_score, average = None, labels = [4]), 
        'recall5': metrics.make_scorer(metrics.recall_score, average = None, labels = [5]),
        'recall6': metrics.make_scorer(metrics.recall_score, average = None, labels = [6])}
    results = cross_validate(model, X, y, cv=CV, scoring=scoring, return_train_score = False)
    for LOS in range(1,7):
        score = results[f'test_recall{LOS}']
        print(f"\tLOS {LOS}: Recall {round(sum(score)/CV*100, 2)}%")

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(model.n_estimators), model.estimator_errors_)

ax.set(xlabel='#Estimator', ylabel='Estimation error',
       title='Evolution of errors training estimators')
ax.grid()
plt.show()

## Feature engineering SAMME

### **FEATURE ENGINEERING ADJACENT LINK INFORMATION**

*Features creation*

For each link, the tt_arrival of the other links are added as features. Therefore we will be using 6 features in total.

In [ ]:
for LINK in range(5,11):
    print(f"LINK {LINK}")
    filename = f'{ROOT}/models/RUSBoost_{PREDICTED_MINUTES_AHEAD}m_SAMME_adjacent_link{LINK}.joblib'

    other_links = [5,6,7,8,9,10]; other_links.remove(LINK)
    training_df_link = training_df.copy()
    training_df_link = training_df_link[training_df_link['link_id'] == LINK]

    for other_link in other_links:
        training_df_link = pd.merge(training_df_link, training_df[training_df['link_id'] == other_link].filter(regex=("tt_ar[^-]+$")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])

    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=60, learning_rate=0.3).fit(training_df_link, "tt_ar[^-]+$", 'LOS_dep')
    joblib.dump(model, filename)

    for LOS in range(1,7):
        validation_df_link = validation_df[(validation_df['link_id'] == LINK) & (validation_df['LOS_dep'] == LOS)].copy()
        for other_link in other_links:
            validation_df_link = pd.merge(validation_df_link, validation_df[validation_df['link_id'] == other_link].filter(regex=("tt_ar[^-]+$")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])
        X_test = validation_df_link.filter(regex=("tt_ar[^-]+$")).values
        y_pred = model.predict(X_test)

        score = metrics.recall_score([LOS]*len(y_pred), y_pred, average='micro')
        print(f"\tLOS {LOS}: Recall {round(score*100, 2) }%")

### **FEATURE ENGINEERING ADJACENT LINK INFORMATION + TEMPORAL INFORMATION (UP TO PREVIOUS 45MIN)**

*Features creation*

For each link, the tt_arrival of the other links, covering the previous 50min from prediction time, are added as features. Therefore we will be using 60 features in total as maximum.

In [ ]:
print(f'USING PREVIOUS {INPUT_MINUTES_BEFORE}m INFORMATION')
for LINK in range(5,11):
    print(f"LINK {LINK}")
    filename = f'{ROOT}/models/RUSBoost_{PREDICTED_MINUTES_AHEAD}m_SAMME_temp{INPUT_MINUTES_BEFORE}m_link{LINK}.joblib'

    other_links = [5,6,7,8,9,10]; other_links.remove(LINK)
    training_df_link = training_df.copy()
    training_df_link = training_df_link[training_df_link['link_id'] == LINK]

    for other_link in other_links:
        training_df_link = pd.merge(training_df_link, training_df[training_df['link_id'] == other_link].filter(regex=("tt_arr.*")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])

    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=60, learning_rate=0.3).fit(training_df_link, "tt_arr.*", 'LOS_dep')
    joblib.dump(model, filename)

    for LOS in range(1,7):
        validation_df_link = validation_df[(validation_df['link_id'] == LINK) & (validation_df['LOS_dep'] == LOS)].copy()
        for other_link in other_links:
            validation_df_link = pd.merge(validation_df_link, validation_df[validation_df['link_id'] == other_link].filter(regex=("tt_arr.*")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])
        X_test = validation_df_link.filter(regex=("tt_arr.*")).values
        y_pred = model.predict(X_test)

        score = metrics.recall_score([LOS]*len(y_pred), y_pred, average='micro')
        print(f"\tLOS {LOS}: Recall {round(score*100, 2) }%")

## Feature engineering M2

### **FEATURE ENGINEERING ADJACENT LINK INFORMATION**

*Features creation*

For each link, the tt_arrival of the other links are added as features. Therefore we will be using 6 features in total.

In [ ]:
for LINK in range(5,11):
    print(f"LINK {LINK}")
    filename = f'{ROOT}/models/RUSBoost_{PREDICTED_MINUTES_AHEAD}m_M2_adjacent_link{LINK}.joblib'

    other_links = [5,6,7,8,9,10]; other_links.remove(LINK)
    training_df_link = training_df.copy()
    training_df_link = training_df_link[training_df_link['link_id'] == LINK]

    for other_link in other_links:
        training_df_link = pd.merge(training_df_link, training_df[training_df['link_id'] == other_link].filter(regex=("tt_ar[^-]+$")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])

    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=60, learning_rate=0.3).fit(training_df_link, "tt_ar[^-]+$", 'LOS_dep')
    joblib.dump(model, filename)

    for LOS in range(1,7):
        validation_df_link = validation_df[(validation_df['link_id'] == LINK) & (validation_df['LOS_dep'] == LOS)].copy()
        for other_link in other_links:
            validation_df_link = pd.merge(validation_df_link, validation_df[validation_df['link_id'] == other_link].filter(regex=("tt_ar[^-]+$")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])
        X_test = validation_df_link.filter(regex=("tt_ar[^-]+$")).values
        y_pred = model.predict(X_test)

        score = metrics.recall_score([LOS]*len(y_pred), y_pred, average='micro')
        print(f"\tLOS {LOS}: Recall {round(score*100, 2) }%")

### **FEATURE ENGINEERING ADJACENT LINK INFORMATION + TEMPORAL INFORMATION (UP TO PREVIOUS 45MIN)**

*Features creation*

For each link, the tt_arrival of the other links, covering the previous 50min from prediction time, are added as features. Therefore we will be using 60 features in total as maximum.

In [ ]:
print(f'USING PREVIOUS {INPUT_MINUTES_BEFORE}m INFORMATION')
for LINK in range(5,11):
    print(f"LINK {LINK}")
    filename = f'{ROOT}/models/RUSBoost_{PREDICTED_MINUTES_AHEAD}m_M2_temp{INPUT_MINUTES_BEFORE}m_link{LINK}.joblib'

    other_links = [5,6,7,8,9,10]; other_links.remove(LINK)
    training_df_link = training_df.copy()
    training_df_link = training_df_link[training_df_link['link_id'] == LINK]

    for other_link in other_links:
        training_df_link = pd.merge(training_df_link, training_df[training_df['link_id'] == other_link].filter(regex=("tt_arr.*")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])

    model = RUSBoostClassifier_(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=60, learning_rate=0.3).fit(training_df_link, "tt_arr.*", 'LOS_dep')
    joblib.dump(model, filename)

    for LOS in range(1,7):
        validation_df_link = validation_df[(validation_df['link_id'] == LINK) & (validation_df['LOS_dep'] == LOS)].copy()
        for other_link in other_links:
            validation_df_link = pd.merge(validation_df_link, validation_df[validation_df['link_id'] == other_link].filter(regex=("tt_arr.*")), on=['date'], how='inner', suffixes=[None, f'_{other_link}'])
        X_test = validation_df_link.filter(regex=("tt_arr.*")).values
        y_pred = model.predict(X_test)

        score = metrics.recall_score([LOS]*len(y_pred), y_pred, average='micro')
        print(f"\tLOS {LOS}: Recall {round(score*100, 2) }%")